In [0]:
from pyspark.sql.functions import col, avg, min, max, year, month, dayofmonth, sum
from pyspark.sql import functions as F

# Leer datos de Silver
df_bronze = spark.table("main.weather.silver_distritos_diarios")

# Conversión de fecha
df = df_bronze.withColumn("fecha", F.to_date("fecha"))

# Tabla Gold principal: Promedios por distrito y día
gold_diaria = df.select(
    "distrito", "fecha", "temperatura_maxima", "temperatura_minima", "precipitacion_total"
)

# Tabla Gold mensual: agregación por distrito y mes
gold_mensual = df.groupBy(
    "distrito",
    year("fecha").alias("anio"),
    month("fecha").alias("mes")
).agg(
    avg("temperatura_maxima").alias("temp_max_promedio"),
    avg("temperatura_minima").alias("temp_min_promedio"),
    sum("precipitacion_total").alias("precipitacion_total_mes")
)

# Persistimos como tablas Gold
gold_diaria.write.format("delta").mode("overwrite").saveAsTable("main.weather.gold_diaria")
gold_mensual.write.format("delta").mode("overwrite").saveAsTable("main.weather.gold_mensual")

print("✅ Capas Gold generadas correctamente.")
